In [9]:
from utils import getStringBetweenTwoWords, extract_and_stringify_object , printJsonToFile
from Product import ProductDetailDTO
#Dependencies
from bs4 import BeautifulSoup
import requests
import json

async def scrape_aliexpress_full(url: str) -> dict:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    script_tag = soup.find('script')
    if script_tag:
        script_content = script_tag.string
        script_target_object= getStringBetweenTwoWords(script_content, 'window.runParams =', 'window.runParams.is23').replace(';', '')
        script_target_object = script_target_object.replace('data', '"data"')
        script_target_object = json.loads(script_target_object)
        price = script_target_object['data']['metaDataComponent']['ogTitle'].split('|')[0].strip()
        title = script_target_object['data']['metaDataComponent']['title'].split('|')[0].strip()
        ImageList = script_target_object['data']['imageComponent']['imagePathList']
        description = script_target_object['data']['productPropComponent']['propGroups']
        attr_values = []
    
        for group in description:
            for prop in group.get("props", []):
                attr_key = prop.get("attrName", "").strip()
                attr_value = prop.get("attrValue", "").strip()
                if attr_value:
                    attr_values.append(attr_key+":"+attr_value)
        
        concatenated_description = ', '.join(attr_values)

    item_data = ProductDetailDTO(name_Global=title, price=price,images=ImageList,productlink1=url,description_Global=concatenated_description)
    return item_data

url = 'https://www.aliexpress.com/item/1005006860843367.html?spm=a2g0o.productlist.main.1.4cbdDLvvDLvvt5&algo_pvid=7246bb87-519c-45f0-9096-37d5a5e6b81c&utparam-url=scene%3Asearch%7Cquery_from%3A&gatewayAdapt=glo2ara'
x=await scrape_aliexpress_full(url)
print("name_global:",x.name_Global)
print("isAvailable:",x.is_available)    
print("price:",x.price)
print("rating:",x.rating)
print("description:",x.description_Global)
print("images:",x.images)




name_global: Apple iPhone 15 Pro ، A3104 ، APPLE ، A17 Pro ، Nano SIM ، IP68 ، إصدار CN ، علامة تجارية جديدة ونشطة ، أصلية ، منتجات أصلية
isAvailable: True
price: 50149.57EGP
rating: 0.0
description: اسم العلامة التجارية:Apple, نموذج وحدة المعالجة المركزية:APPLE, عرض الحجم:6.6, دقة شاشة العرض:2556x1179, معدل تحديث الشاشة:120Hz, Touch Screen:نعم, مواد الشاشة:OLED, Screen Type:كامل الشاشة, حجم الكاميرا الخلفية:3, كمية الكاميرا الأمامية:1, دقة الكاميرا الخلفية:12 ميجا بكسل,48 ميجا بكسل, بكسل الكاميرا الأمامية:12 ميجا بكسل, Battery Capacity(mAh):3200, Battery Type:غير قابل للانفصال, الشحن اللاسلكي:YES, الشحن السريع:USB-PD, طاقة الشحن:أخرى, نوع واجهة الشحن:TYPE-C, منفذ سماعة رأس 3.5 ملم:no, نوع بطاقة الذاكرة:غير معتمدة, SIM عدد بطاقة:2 بطاقتين شريحة SIM, IP68/IP69K:YES, نظام التشغيل:IOS, الجوال:5 G, NFC:YES, نسخة البلوتوث:YSE, واي فاي:YES, المدمج في GPS:YES, الميزات الأخرى:البصرية الاستقرار,صورة الهاتف,5G الهاتف المحمول,سريع شحن الهاتف, تكنولوجيا القياسات الحيوية:التعرف على الوجه, معيار نقل